In [2]:
import numpy as np

# Colors
WHITE = 0
BLUE = 1
RED = 2
GREEN = 3  # Initial piece
COLORS = np.array([WHITE, BLUE, RED])

In [3]:
# Shapes
LONG4 = np.array((1, 1, 4))
RECTANGLE3 = np.array((2, 1, 3))
RECTANGLE2 = np.array((3, 1, 2))
SQUARE2 = np.array((2, 1, 2))
SHORT2 = np.array((1, 1, 2))
LONG1 = np.array((4, 1, 1))
RECTANGLE1 = np.array((3, 2, 1))
SQUARE1 = np.array((2, 2, 1))
SHORT1 = np.array((2, 1, 1))
SHAPES = np.array([LONG4, RECTANGLE3, RECTANGLE2, SQUARE2, SHORT2, LONG1,
                   RECTANGLE1, SQUARE1, SHORT1])

In [4]:
# Can we do that better ?
PIECES=np.column_stack((np.repeat(SHAPES, len(COLORS), axis=0), np.tile(COLORS, len(SHAPES))))
IDX, IDY, IDZ, IDCOL = 0, 1, 2, 3
IDPX, IDPY, IDPZ = 4, 5, 6

In [5]:
ABC = (RECTANGLE1, False)
ACB = (RECTANGLE2, False)
BAC = (RECTANGLE1, True)
BCA = (RECTANGLE3, False)
CAB = (RECTANGLE2, True)
CBA = (RECTANGLE3, True)

In [6]:
CARD_WHITE = PIECES[PIECES[:,IDCOL] == WHITE]
CARD_BLUE = PIECES[PIECES[:,IDCOL] == BLUE]
CARD_RED = PIECES[PIECES[:,IDCOL] == RED]

CARD_LONG = PIECES[np.all(np.sort(PIECES[:,:3], axis=1)[:,:2] == np.array((1, 1)), axis=1)]
CARD_SQUARE = PIECES[np.all(np.sort(PIECES[:,:3], axis=1)[:,1:] == np.array((2, 2)), axis=1)]
CARD_RECTANGLE = PIECES[np.all(np.sort(PIECES[:,:3], axis=1) == np.array((1, 2, 3)), axis=1)]

#CARD_RECTANGLE = frozenset(p for p in PIECES if sorted(p.shape.value) == [1, 2, 3])

In [281]:

# Remember to remove the rectangle
INITIAL_DECK = np.array([CARD_SQUARE, CARD_WHITE, CARD_LONG, CARD_RECTANGLE, CARD_BLUE,
                         CARD_RED, CARD_WHITE, CARD_RECTANGLE, CARD_WHITE, CARD_BLUE,
                         CARD_RED, CARD_SQUARE, CARD_LONG, CARD_RECTANGLE, CARD_BLUE,
                         CARD_LONG, CARD_LONG, CARD_BLUE, CARD_SQUARE, CARD_RED, CARD_LONG,
                         CARD_BLUE, CARD_RED, CARD_LONG, CARD_RECTANGLE, CARD_SQUARE,
                         CARD_WHITE, CARD_BLUE, CARD_WHITE, CARD_WHITE, CARD_RED, CARD_RED,
                         CARD_RECTANGLE, CARD_RECTANGLE, CARD_SQUARE, CARD_SQUARE])

CARDS_NUM = len(INITIAL_DECK)

In [8]:
BOARD = np.array((6, 4))

In [364]:
import matplotlib.pyplot as plt
import numpy as np

# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
    
def draw(pieces):
    pieces = np.copy(pieces)
    # shape for pieces is (n, 7), 7 being (l, w, h, c, x, y, z)
    idl, idw, idh, idc, idx, idy, idz = range(7)
    pieces[:,idz] += 1
    max_height = np.max(pieces[:,idh] + pieces[:,idz]) + 1
    
    # prepare some coordinates
    shape = np.concatenate((BOARD, np.array((max_height,))))
    x, y, z = np.indices(shape)
    colors = np.full(shape, "")
    color_names = ["white", "blue", "red", "green"]
    
    base = (z == 0)
    colors[base] = 'key'
    
    for piece in pieces:
        voxels = ((piece[idx] <= x) & (x < piece[idx] + piece[idl]) &
                  (piece[idy] <= y) & (y < piece[idy] + piece[idw]) &
                  (piece[idz] <= z) & (z < piece[idz] + piece[idh]))

        colors[voxels] = color_names[piece[idc]]

    voxels = colors != ""

    # and plot everything
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')
    ax.voxels(voxels, facecolors=colors, edgecolor='#00000020')
    set_axes_equal(ax)
    plt.axis('off')
    plt.show()



In [365]:
import functools

def concatenate_two(a, b):
    return np.hstack((np.repeat(a, len(b), axis=0), np.tile(b, (len(a), 1))))

def combine(*arrays):
    return functools.reduce(concatenate_two, arrays)

In [455]:
from typing import Sequence
import numpy as np


class Game:
    def __init__(self, turn: int, deck, used, board_cubes, board_pieces, first):
        assert turn < CARDS_NUM
        self.turn = turn
        self.deck = deck
        self.used = used
        self.board_cubes = board_cubes
        self.board_pieces = board_pieces
        self.first = first

    @classmethod
    def new(cls, first, cards_order=None):
        first_shape, rotated = first
        if rotated:
            first_shape = first_shape[[1, 0, 2]]

        sorted_cards = np.arange(CARDS_NUM)
        if cards_order:
            assert np.sort(cards_order) == sorted_cards
        else:
            cards_order = sorted_cards
        deck = INITIAL_DECK[cards_order]

        board_cubes = np.zeros(BOARD, dtype=int)
        board_cubes[: first_shape[IDX], : first_shape[IDY]] = first_shape[IDZ]
        board_pieces = np.array([[*first_shape, GREEN, 0, 0, 0]])
        return cls(
            turn=0,
            deck=deck,
            used=np.empty((0, 4)),
            board_cubes=board_cubes,
            board_pieces=board_pieces,
            first=first,
        )

    def draw(self):
        draw(pieces=self.board_pieces)

    def possible_moves(self):
        # All possible blocks from the card
        authorized = self.deck[self.turn]

        # Remove those already used
        available = authorized[
            np.all(np.any(authorized != self.used[:, None, :], axis=2), axis=0)
        ]
        if not available.shape[0]:
            return available
        
        # Add all possible rotations
        avail_and_rotated = np.unique(
            np.concatenate((available, available[:, (1, 0, 2, 3)])), axis=0
        )

        # Combine with every possible coordinate
        moves = combine(
            avail_and_rotated,
            np.arange(BOARD[0])[..., None],
            np.arange(BOARD[1])[..., None],
        )

        # Remove out of bounds
        moves = moves[
            ~np.any(
                [
                    moves[:, IDX] + moves[:, IDPX] > BOARD[0],
                    moves[:, IDY] + moves[:, IDPY] > BOARD[1],
                ],
                axis=0,
            )
        ]
        if not moves.shape[0]:
            return moves

        # Remove z clash
        x, y = np.indices(BOARD)
        # This will be the list of coordinates for each move
        coord = moves[:, (IDPX, IDPY)]
        # This is board cubes, repeated for each move
        repeated = np.repeat(self.board_cubes[None], len(moves), axis=0)
        # This is the height of the point a each coordinate
        heights = self.board_cubes[tuple(coord.T)]
        # This is a mask the shape of the board, with True where the piece is, for each move
        shapes = np.moveaxis(
            np.all(
                [
                    moves[:, IDPX] <= x[..., None],
                    x[..., None] < moves[:, IDPX] + moves[:, IDX],
                    moves[:, IDPY] <= y[..., None],
                    y[..., None] < moves[:, IDPY] + moves[:, IDY],
                ],
                axis=0,
            ),
            -1,
            0,
        )

        # This is a mask the shape of the board with all the places where height is equal to
        # the height at the coordinate
        equal = np.equal(repeated, heights[..., None, None])

        # Couldn't find how to remove the for loop here :(
        # Here we keep only the pieces for which equal[shape] is all True
        moves_to_keep = np.array([np.all(e[s]) for e, s in zip(equal, shapes)])
        moves = moves[moves_to_keep]
        heights = heights[moves_to_keep]
        shapes = shapes[moves_to_keep]
        if not moves.shape[0]:
            return moves

        # Get 3D coordinates (same format as board_pieces)
        moves = np.concatenate([moves, heights[..., None]], axis=1)
        
        # pairwise differences
        diff = moves[None] - self.board_pieces[:, None]

        same_shape = np.all(diff[..., 0:3] == 0, axis=2)

        # 2 pieces are close if they have non zero diff distance on 1 coordinate, and that this
        # non zero (absolute) value is the size of the piece
        relative_dist = np.abs(diff[..., IDPX:]) / moves[..., IDX : IDZ + 1]
        close = np.all(np.sort(relative_dist, axis=2) == np.array([0, 0, 1]), axis=2)

        both = np.any(same_shape & close, axis=0)

        moves = moves[~both]
        shapes = shapes[~both]

        # Keep only moves that are touching another piece
        possible_spots = self.board_cubes.copy()
        mask = np.copy(possible_spots)
        possible_spots[:-2, :] += mask[1:-1, :]
        possible_spots[2:, :] += mask[1:-1:, :]
        possible_spots[:, :-2] += mask[:, 1:-1]
        possible_spots[:, 2:] += mask[:, 1:-1]
        possible_spots = possible_spots > 0
        block_is_next_to_piece = shapes & possible_spots
        is_next_to_piece = np.any(block_is_next_to_piece, axis=(1, 2))

        moves = moves[is_next_to_piece]
        
        return moves

    def play(self, move):
        
        
        board_pieces = np.append(self.board_pieces, move[None], axis=0)
        board_cubes = np.copy(self.board_cubes)
        board_cubes[
            move[IDPX] : move[IDPX] + move[IDX], move[IDPY] : move[IDPY] + move[IDY]
        ] += move[IDZ]

        rot = move[IDX] < move[IDY]
        if rot:
            move[[IDX, IDY]] = move[[IDY, IDX]]
        
        used = np.append(self.used, move[IDX : IDCOL + 1][None], axis=0)
        
        return self.__class__(
            turn=self.turn + 1,
            deck=self.deck,
            used=used,
            board_cubes=board_cubes,
            board_pieces=board_pieces,
            first=self.first,
        )
    @property
    def score(self):
        return np.min(self.board_cubes)
    

In [456]:
def random_game(game: Game):
    game = 
    while True:
        moves = game.possible_moves()
        if not len(moves):
            # game.draw()
            return game.score
        else:
            game = game.play(moves[np.random.randint(moves.shape[0])])

In [448]:
%timeit random_game()

18 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [425]:
def count_nodes(level:int, game:Game, total_moves:list):
    if level < 0:
        return
    possible_moves = game.possible_moves()
    total_moves[level] += len(possible_moves)
    for move in possible_moves:
        count_nodes(level-1, game.play(move), total_moves)

In [428]:
l = [0] * 5
count_nodes(2, Game.new(ABC), l)

In [457]:
scores = np.array([random_game() for _ in range(1000)])

In [458]:
np.average(scores)

0.022

In [459]:
np.max(scores)

1